# CS 229 Final Project Notebook 

### Overview

Implementation of LeNet-5 and similar architectures to evaluate complex- and split-complex-valued neural networks. Network is implemented in TensorFlow, and wrappers for complex and split-complex valued implementations are given in layersHDA.py ("layers high-dimensional algebra"). 

### High-dimensional Algebras

To implement layers with complex- and split-complex-valued weights, we can store the weights for the real/imaginary channels separately and use weight-sharing between operations. This is implemented with wrapper files for TensorFlow commands. 

### References
Code implemented here was based on open source code available at: https://github.com/sujaybabruwad/LeNet-in-Tensorflow. The functions to extract CIFAR-10, CIFAR-100, and SVHN were based on code available at: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py.


In [1]:
# Front matter: load libraries needed for training
import numpy as np
import tensorflow as tf
from utils import *

import os

%reload_ext autoreload
%autoreload 2

In [2]:
# Runfile preliminaries
networkSizes = ['Baseline', 'Wide','Deep']
networkTypes = ['Real', 'Complex','SplitComplex']

# MNIST Training

The cells below load MNIST data and execute run files for the various network architectures. Each iteration saves loss/accuracy curves for the training, validation, and test sets, and the model at the end of training. Models are trained for 20 epochs with a batch size of 200. 

In [22]:
# Initialize MNIST Data

dataset = 'MNIST' #set book-keeping parameter

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", reshape=False)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

# Zero-pad MNIST images to be 32x32
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Updated Image Shape: (32, 32, 1)


In [25]:
# Set training parameters
EPOCHS = 40
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 

In [28]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), 
                 dataset, BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-3)

Training...

EPOCH 1 ...
Validation Accuracy = 0.939

EPOCH 2 ...
Validation Accuracy = 0.962

EPOCH 3 ...
Validation Accuracy = 0.973

EPOCH 4 ...
Validation Accuracy = 0.981

EPOCH 5 ...
Validation Accuracy = 0.984

EPOCH 6 ...
Validation Accuracy = 0.985

EPOCH 7 ...
Validation Accuracy = 0.985

EPOCH 8 ...
Validation Accuracy = 0.986

EPOCH 9 ...
Validation Accuracy = 0.987

EPOCH 10 ...
Validation Accuracy = 0.987

EPOCH 11 ...
Validation Accuracy = 0.984

EPOCH 12 ...
Validation Accuracy = 0.985

EPOCH 13 ...
Validation Accuracy = 0.987

EPOCH 14 ...
Validation Accuracy = 0.988

EPOCH 15 ...
Validation Accuracy = 0.988

EPOCH 16 ...
Validation Accuracy = 0.988

EPOCH 17 ...
Validation Accuracy = 0.988

EPOCH 18 ...
Validation Accuracy = 0.989

EPOCH 19 ...
Validation Accuracy = 0.988

EPOCH 20 ...
Validation Accuracy = 0.988

EPOCH 21 ...
Validation Accuracy = 0.987

EPOCH 22 ...
Validation Accuracy = 0.988

EPOCH 23 ...
Validation Accuracy = 0.989

EPOCH 24 ...
Validation Accura

EPOCH 26 ...
Validation Accuracy = 0.988

EPOCH 27 ...
Validation Accuracy = 0.991

EPOCH 28 ...
Validation Accuracy = 0.992

EPOCH 29 ...
Validation Accuracy = 0.988

EPOCH 30 ...
Validation Accuracy = 0.989

EPOCH 31 ...
Validation Accuracy = 0.990

EPOCH 32 ...
Validation Accuracy = 0.990

EPOCH 33 ...
Validation Accuracy = 0.988

EPOCH 34 ...
Validation Accuracy = 0.992

EPOCH 35 ...
Validation Accuracy = 0.991

EPOCH 36 ...
Validation Accuracy = 0.989

EPOCH 37 ...
Validation Accuracy = 0.991

EPOCH 38 ...
Validation Accuracy = 0.990

EPOCH 39 ...
Validation Accuracy = 0.992

EPOCH 40 ...
Validation Accuracy = 0.991

Model saved
INFO:tensorflow:Restoring parameters from ./lenet_MNIST_Wide_Complex
Test Accuracy = 0.990
Training...

EPOCH 1 ...
Validation Accuracy = 0.981

EPOCH 2 ...
Validation Accuracy = 0.984

EPOCH 3 ...
Validation Accuracy = 0.985

EPOCH 4 ...
Validation Accuracy = 0.989

EPOCH 5 ...
Validation Accuracy = 0.987

EPOCH 6 ...
Validation Accuracy = 0.987

EPOCH 7 

# CIFAR-10 Training
Cells below load the CIFAR-10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html), setup the training parameters, and make calls to train the 9 models we are interested in. 

In [16]:
# Initialize CIFAR-10 Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import cifar10
dataset = 'CIFAR10' #set book-keeping parameter

train_tup, test_tup = cifar10.load_data()

X_train, y_train = train_tup
X_validation = X_train[49000:,:]
y_validation = y_train[49000:]
X_train = X_train[:49000,:]
y_train = y_train[:49000]

X_test, y_test = test_tup

In [18]:
# Set training parameters
EPOCHS = 50
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

Number of training examples: 49000
Number of validation examples: 1000
Number of testing examples: 10000


In [24]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-3)

Training...

EPOCH 1 ...
Validation Accuracy = 0.444

Model saved
INFO:tensorflow:Restoring parameters from ./lenet_CIFAR10_Baseline_Real
Test Accuracy = 0.434
Training...



KeyboardInterrupt: 

# CIFAR-100 Training
Cells below load the CIFAR-100 dataset (https://www.cs.toronto.edu/~kriz/cifar.html), setup the training parameters, and make calls to train the 9 models we are interested in. (This code is nearly identical to the CIFAR-10 cells above.)

In [19]:
# Initialize CIFAR-100 Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import cifar100
dataset = 'CIFAR100' #set book-keeping parameter

train_tup, test_tup = cifar100.load_data()

X_train, y_train = train_tup
X_validation = X_train[49000:,:]
y_validation = y_train[49000:]
X_train = X_train[:49000,:]
y_train = y_train[:49000]

X_test, y_test = test_tup

In [35]:
# Set training parameters
EPOCHS = 50
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

Number of training examples: 49000
Number of validation examples: 1000
Number of testing examples: 10000


In [37]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 100, networkSize = networkSize, networkType = networkType, alpha = 1e-3)

Training...

EPOCH 1 ...
Validation Accuracy = 0.080

Model saved
INFO:tensorflow:Restoring parameters from ./lenet_CIFAR100_Baseline_Real
Test Accuracy = 0.098
Training...



KeyboardInterrupt: 

# SVHN Training
Cells below load the SVHN dataset (http://ufldl.stanford.edu/housenumbers/), setup the training parameters, and make calls to train the 9 models we are interested in. (This code is nearly identical to the CIFAR-10 cells above.)

In [58]:
# Initialize SVHN Data
# Used code from: https://github.com/tflearn/tflearn/blob/master/tflearn/datasets/cifar10.py
import svhn
dataset = 'SVHN' #set book-keeping parameter

train_tup, test_tup = svhn.load_data()

X_train, y_train = train_tup
X_validation = X_train[66257:,:]
y_validation = y_train[66257:]
X_train = X_train[:66257,:]
y_train = y_train[:66257]

X_test, y_test = test_tup
y_test = y_test

In [59]:
y = tf.placeholder(tf.int32, (None))
y_one_hot = tf.one_hot(y,10)
y_argmax = tf.argmax(y_one_hot,dimension=1)

with tf.Session() as sess:
    yout = sess.run(y_one_hot, feed_dict = {y: y_test[0:10]})
    print(yout)
    yout = sess.run(y_argmax, feed_dict = {y: y_test[0:10]})
    print(yout)

print(y_test[0:10])
print(y_test[0])

[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]]
[5 2 1 0 6 1 9 1 1 8]
[5, 2, 1, 0, 6, 1, 9, 1, 1, 8]
5


In [60]:
# Set training parameters
EPOCHS = 1
BATCH_SIZE = 200
NUM_TRAIN = X_train.shape[0] # set number of training examples
NUM_VAL = X_validation.shape[0] 
NUM_TEST = X_test.shape[0] 
print('Number of training examples: {}'.format(NUM_TRAIN))
print('Number of validation examples: {}'.format(NUM_VAL))
print('Number of testing examples: {}'.format(NUM_TEST))

Number of training examples: 66257
Number of validation examples: 7000
Number of testing examples: 26032


In [62]:
# Loop over sizes and types to generate data

for networkSize in networkSizes:
    for networkType in networkTypes:
        runmodel((X_train,y_train,NUM_TRAIN), (X_validation,y_validation,NUM_VAL), (X_test,y_test,NUM_TEST), dataset, 
                 BATCH_SIZE, EPOCHS, num_classes = 10, networkSize = networkSize, networkType = networkType, alpha = 1e-3)

Training...

EPOCH 1 ...
Validation Accuracy = 0.181



KeyboardInterrupt: 